In [ ]:
import pandas as pd
from selenium import webdriver 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options
import time, traceback
from multiprocessing.dummy import Pool as ThreadPool
from multiprocessing import Pool
from pathos.multiprocessing import ProcessingPool as Pool2
import threading

In [ ]:
def startDriver(file_path):
    option = webdriver.ChromeOptions()
    option.add_argument(" — incognito")
    browser = webdriver.Chrome(executable_path=file_path)
    browser.get("https://coinmarketcap.com/exchanges/bitmax/")
    return browser

In [ ]:
browser = startDriver("C:\\Users\\Michał\\GIT\\CryptocurrencyArbitrage\\chromedriver.exe")


In [ ]:
def getRates(browser):
    exchange_lists = []
    curr_tab = browser.find_elements_by_xpath('//table[@id="exchange-markets"]/tbody/tr')
    for tr in curr_tab:
        single_exchange = []
        td = tr.find_elements_by_tag_name("td")
        for i in range(1,6):
            if len(td[i].find_elements_by_class_name("price")) == 0:
                single_exchange.append(td[i].get_attribute("data-sort"))
            else:
                single_exchange.append(td[i].find_element_by_class_name("price").get_attribute("data-native"))
                single_exchange.append(td[i].get_attribute("data-sort"))
        exchange_lists.append(single_exchange)
    df = pd.DataFrame(exchange_lists, columns = ["Currency", "Pair", "Volume", "Rate", "PriceUSD", "VolumePerc"])
    df[['Numerator','Denumerator']] = df["Pair"].str.split("/",expand=True,)
    
    print("Scraped", time.time())

In [ ]:
def refreshRates(delay, task):
    next_time = time.time() + delay
    while True:
        time.sleep(max(0, next_time - time.time()))
        try:
          task
        except Exception:
          traceback.print_exc()
        next_time += (time.time() - next_time) // delay * delay + delay

In [ ]:

# threading.Thread(target=lambda: refreshRates(5, getRates(browser))).start()

In [ ]:
refreshRates(30, getRates(browser))